<a href="https://colab.research.google.com/github/caioitalo/soulcode/blob/main/Pr%C3%A1tica_MongoDB_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Utilizando um Banco de dados base do próprio MongoDB foi feito um ETL sobre o dataset, utilizando conhecimentos de MongoDB e Pandas.**

# EXTRAÇÃO DOS DADOS

In [1]:
!pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 122 kB 4.8 MB/s 


In [2]:
import pandas as pd
import pandera as pa
import numpy as np

## Importação do dataset a partir de certificado do MongoDB

In [3]:
from pymongo import MongoClient

uri = "mongodb+srv://caio-soulcode.o09ab0z.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri, tls=True, tlsCertificateKeyFile='/content/X509-cert-9074189039604702649.pem')

db = client['sample_supplies']
colecao = db['sales']


In [4]:
cursor = colecao.find({})
df = pd.DataFrame(list(cursor))

## Backup do Data Frame

In [5]:
dfback = df.copy()

In [ ]:
# use somente caso dê erro no dataframe
df = dfback.copy()

#TRANSFORMAÇÃO DOS DADOS

In [28]:
df.head(2)

,id_cli,data_da_venda,carrinho,cidade_loja,cliente,usou_cupom,forma_compra,sexo,idade,email,satisfacao
0,0,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4
1,1,2015-08-25 10:01:02.918,"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",False,Phone,M,50,keecade@hem.uy,5


## Drop de coluna do ID do documento do MongoDB

In [6]:
df.drop('_id', axis=1, inplace=True)

## Percorrer a coluna customer e separar os dados em outras colunas

In [8]:
age = []
email = []
sat = []
gen = []

for i in range(len(df)):
  gen.append(str(df.customer[i]['gender']))
  age.append(int(df.customer[i]['age']))
  email.append(str(df.customer[i]['email']))
  sat.append(str(df.customer[i]['satisfaction']))

df['sexo'] = gen
df['idade'] = age
df['email'] = email
df['satisfacao'] = sat

## Renomeação das colunas para português

In [9]:
df.rename(columns={'saleDate':'data_da_venda','items':'carrinho','storeLocation':'cidade_loja','customer':'cliente','couponUsed':'usou_cupom',
                   'purchaseMethod':'forma_compra'},inplace=True)

In [38]:
df.dtypes

id_cli                    int64
data_da_venda    datetime64[ns]
carrinho                 object
cidade_loja              object
cliente                  object
usou_cupom                 bool
forma_compra             object
sexo                     object
idade                     int64
email                    object
satisfacao               object
dtype: object

## Inserindo uma coluna de ID de cliente, depois atribuindo valores e por fim tornando-os do tipo inteiro.

In [10]:
df.insert(0,'id_cli',np.nan)

In [11]:
for i in range(len(df)):
  df['id_cli'][i]= i

<ipython-input-11-a62897187e97>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id_cli'][i]= i


In [12]:
df.id_cli = df.id_cli.astype('int')

## Verificação de dados faltantes

In [ ]:
df.isna().sum()

data_da_venda    0
carrinho         0
cidade_loja      0
cliente          0
usou_cupom       0
forma_compra     0
sexo             0
idade            0
email            0
satisfacao       0
dtype: int64

## Checagem de todas as colunas para observar possíveis inconsistências

In [13]:
# checando os valores das colunas
df.groupby(['email']).size().sort_values(ascending=False)

email
ab@bitropot.bg       1
paujzi@fihuglu.sa    1
pe@ce.cz             1
pe@cawsenmo.aw       1
pe@bo.gn             1
                    ..
ho@hi.va             1
ho@en.vu             1
ho@ekjiknev.gov      1
ho@dikajvo.mk        1
zuzlavveb@ted.gp     1
Length: 5000, dtype: int64

## Criação de outro Dataframe somente com as informações das compras e com ID do cliente

In [23]:
df2 = pd.DataFrame()
df2.insert(0,'id_compra',np.nan)
df2.insert(1,'id_cli',np.nan)
df2.insert(2,'nome_prod',np.nan)
df2.insert(3,'qtd',np.nan)
df2.insert(4,'preco',np.nan)
df2.insert(5,'tags',np.nan)

In [24]:
idcli = []
nomeprod =[]
preco = []
qtd=[]
tag=[]
idcompra = []
cont = 0

for i in range(len(df)):
  for c in range(len(df.carrinho[i])):
    idcompra.append(cont)
    idcli.append(i)
    nomeprod.append(df.carrinho[i][c]['name'])
    preco.append(df.carrinho[i][c]['price'])
    qtd.append(df.carrinho[i][c]['quantity'])
    tag.append((str(df.carrinho[i][c]['tags'])))
    cont += 1

df2['id_compra'] = idcompra
df2['id_cli'] = idcli
df2['nome_prod'] = nomeprod
df2['preco'] = preco
df2['qtd'] = qtd
df2['tags'] = tag

## Últimos ajustes no primeiro Dataframe


In [30]:
df.drop(columns=['carrinho','cliente'],inplace=True)

In [31]:
df

,id_cli,data_da_venda,cidade_loja,usou_cupom,forma_compra,sexo,idade,email,satisfacao
0,0,2015-03-23 21:06:49.506,Denver,True,Online,M,42,cauho@witwuta.sv,4
1,1,2015-08-25 10:01:02.918,Seattle,False,Phone,M,50,keecade@hem.uy,5
2,2,2017-06-22 09:54:14.185,Denver,False,In store,M,51,worbiduh@vowbu.cg,5
3,3,2015-02-23 09:53:59.343,Seattle,False,In store,F,45,vatires@ta.pe,3
4,4,2017-12-03 18:39:48.253,London,False,In store,M,40,dotzu@ib.sh,4
...,...,...,...,...,...,...,...,...,...
4995,4995,2016-01-20 17:29:10.225,Austin,False,In store,F,45,du@rum.py,3
4996,4996,2014-11-29 10:09:41.573,Denver,False,In store,F,47,dup@katjewtep.hr,3
4997,4997,2017-12-25 07:02:51.283,Denver,False,In store,M,29,losuhse@eronehjod.np,4
4998,4998,2013-12-07 11:06:25.510,Denver,False,Online,M,59,hu@pehabur.gd,4


In [25]:
df2

,id_compra,id_cli,nome_prod,qtd,preco,tags
0,0,0,printer paper,2,40.01,"['office', 'stationary']"
1,1,0,notepad,2,35.29,"['office', 'writing', 'school']"
2,2,0,pens,5,56.12,"['writing', 'office', 'school', 'stationary']"
3,3,0,backpack,2,77.71,"['school', 'travel', 'kids']"
4,4,0,notepad,2,18.47,"['office', 'writing', 'school']"
...,...,...,...,...,...,...
27433,27433,4999,envelopes,5,24.12,"['stationary', 'office', 'general']"
27434,27434,4999,pens,4,27.1,"['writing', 'office', 'school', 'stationary']"
27435,27435,4999,binder,3,29.09,"['school', 'general', 'organization']"
27436,27436,4999,laptop,2,598.79,"['electronics', 'school', 'office']"


In [22]:
df2.groupby(['tag1']).size().sort_values(ascending=False)

tag1
['office', 'writing', 'school']                  6872
['school', 'general', 'organization']            4676
['writing', 'office', 'school', 'stationary']    4559
['stationary', 'office', 'general']              4505
['school', 'travel', 'kids']                     2309
['electronics', 'school', 'office']              2271
['office', 'stationary']                         2246
dtype: int64